In [21]:
import requests
from dataclasses import dataclass
from typing import List, Optional
from dotenv import load_dotenv
import os

load_dotenv()


True

In [22]:
headers = {'Accept': 'application/json'}
response = requests.get(os.getenv('URL_PATH'), headers=headers)

In [ ]:
@dataclass
class Rating:
    score: float
    count: int

@dataclass
class Stats:
    userRating: Rating
    pressRating: Rating

@dataclass
class Projection:
    cinema_Id: str
    cinema_Name: str
    showTimes: List[str]

@dataclass
class Movie:
    movieId: str
    synopsis: str
    poster: str
    title: str
    runtime: int
    genres: List[str]
    languages: List[str]
    productionYear: int
    stats: Stats
    projectionDubbed: Projection
    projectionOriginal: Projection

In [17]:
def parse_movie_data(element):
    movie_info = {element['movie']['id'] : {
                        'synopsis':element['movie']['synopsis'],
                        'posterUrl':element['movie']['poster']['url'],
                        'title': element['movie']['title'],
                        'runtime': element['movie']['runtime'],
                        'genre': [ele['translate'] for ele in element['movie']['genres']],
                        'langages' : [ele for ele in element['movie']['languages']],
                        'stats' : {'userRating':{'score':element['movie']['stats']['userRating']['score'], 
                                                 'count':element['movie']['stats']['userRating']['count']}, 
                                   'pressRating':{'score':element['movie']['stats']['pressReview']['score'], 
                                                  'count':element['movie']['stats']['pressReview']['count']}},
                        'certificate': element['movie']['releases'][0]['certificate']['label'],
                        'credits': [{'lastName':ele['person']['lastName'],
                                       'firstName':ele['person']['firstName'],
                                       'pictureUrl':ele['person']['picture']['url'],
                                       'position': ele['position']['name']}
                                      for ele in element['movie']['credits']],
                        'actors': [{'lastName':ele['node']['actor']['lastName'],
                                    'firstName':ele['node']['actor']['firstName'],
                                    'pictureUrl':ele['node']['actor'].get('picture',{}).get('url'),
                                    'position': 'actor'}
                                    for ele in element['movie']['cast']['edges']],
                        }}
    return movie_info

In [19]:
from dataclasses import dataclass
from typing import List, Optional, Dict

@dataclass
class Rating:
    score: Optional[float] = None
    count: Optional[int] = None

@dataclass
class Stats:
    userRating: Rating
    pressRating: Rating

@dataclass
class Person:
    lastName: str
    firstName: str
    pictureUrl: Optional[str] = None
    position: str

@dataclass
class Movie:
    id: str
    title: str
    synopsis: str
    posterUrl: Optional[str] = None
    runtime: int
    genre: Optional[List[str]] = None
    languages: Optional[List[str]] = None
    stats: Optional[Stats] = None
    certificate: Optional[str] = None
    credits: Optional[List[Person]] = None
    actors: Optional[List[Person]] = None

def get_url_from_nested(data: dict, *keys) -> Optional[str]:
    """Extract URL from nested dictionary structure."""
    current = data
    for key in keys:
        if not isinstance(current, dict):
            return None
        current = current.get(key)
        if current is None:
            return None
    return current

def parse_person(data: dict, is_actor: bool = False) -> Person:
    """Parse person data into Person object."""
    if is_actor:
        base = data.get('node', {}).get('actor', {})
    else:
        base = data.get('person', {})
    
    return Person(
        lastName=base.get('lastName'),
        firstName=base.get('firstName'),
        pictureUrl=get_url_from_nested(base, 'picture', 'url'),
        position='actor' if is_actor else data.get('position', {}).get('name')
    )

def parse_stats(stats_data: Optional[dict]) -> Optional[Stats]:
    """Parse stats data into Stats object."""
    if not stats_data:
        return None
    
    return Stats(
        userRating=Rating(
            score=stats_data.get('userRating', {}).get('score'),
            count=stats_data.get('userRating', {}).get('count')
        ),
        pressRating=Rating(
            score=stats_data.get('pressReview', {}).get('score'),
            count=stats_data.get('pressReview', {}).get('count')
        )
    )

def parse_movie_data(element: dict) -> Dict[str, Movie]:
    """Parse movie data into Movie object."""
    movie = element.get('movie', {})
    movie_id = movie.get('id', 'unknown_id')
    
    return {movie_id: Movie(
        id=movie_id,
        title=movie.get('title'),
        synopsis=movie.get('synopsis'),
        posterUrl=get_url_from_nested(movie, 'poster', 'url'),
        runtime=movie.get('runtime'),
        genre=[ele.get('translate') for ele in movie.get('genres', [])] if movie.get('genres') else None,
        languages=movie.get('languages'),
        stats=parse_stats(movie.get('stats')),
        certificate=get_url_from_nested(movie, 'releases', 0, 'certificate', 'label'),
        credits=[parse_person(ele) for ele in movie.get('credits', [])] if movie.get('credits') else None,
        actors=[parse_person(ele, is_actor=True) for ele in movie.get('cast', {}).get('edges', [])]
            if movie.get('cast', {}).get('edges') else None
    )}

# Usage
structured_output = []
for element in response.json()["results"]:
    movie_info = parse_movie_data(element)
    structured_output.append(movie_info)

In [20]:
structured_output

[{'TW92aWU6MzA0NDA5': Movie(id='TW92aWU6MzA0NDA5', title='Hiver à Sokcho', synopsis='A Sokcho, petite ville balnéaire de Corée du Sud, Soo-Ha, 23 ans, mène une vie routinière, entre ses visites à sa mère, marchande de poissons, et sa relation avec son petit ami, Jun-oh. L’arrivée d’un Français, Yan Kerrand, dans la petite pension dans laquelle Soo-Ha travaille, réveille en elle des questions sur sa propre identité et sur son père français dont elle ne sait presque rien. Tandis que l’hiver engourdit la ville, Soo-Ha et Yan Kerrand vont s’observer, se jauger, tenter de communiquer avec leurs propres moyens et tisser un lien fragile.', posterUrl='https://fr.web.img3.acsta.net/img/04/d7/04d71113c83535c44e8924e6e948275c.jpg', runtime='1h 45min', genre=['Drame'], languages=['FRENCH', 'KOREAN'], stats=Stats(userRating=Rating(score=3.81, count=309), pressRating=Rating(score=3.17, count=29)), certificate=None, credits=[Person(lastName='Kamura', firstName='Koya', pictureUrl='https://fr.web.img2.